In [ ]:
#13.1.1 Think Step by Step

from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

prompt_template = "I went to the market and bought 20 mangoes.I gave 5 apples to the Ram and 3 to the House-help. I then went and bought 12 more mangoes and ate 2.How many mangoes did I remain with? Let's think step by step."

llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))


In [ ]:
llm_chain({})

{'text': '\n\nStep 1: I bought 20 mangoes.\nStep 2: I gave away 5 mangoes to Ram and 3 mangoes to the House-help.\nStep 3: I bought 12 more mangoes.\nStep 4: I ate 2 mangoes.\n\nAnswer: I remained with 30 mangoes.'}

In [ ]:
#13.1.2 Few-Shot Prompting

from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
    {
        "Question": "Yes or no: Would a rubber duck sink in water?",
        "Reasoning": "The density of a typical rubber duck is less than water, typically around 0.3 g/cm³. Therefore, a rubber duck would float.",
        "Answer": "No"
    },
    {
        "Question": "Yes or no: Would a wooden block sink in water?",
        "Reasoning": "The density of wood is generally less than water, averaging around 0.6 to 0.9 g/cm³. Therefore, a wooden block would likely float.",
        "Answer": "No"
    },
    {
        "Question": "Yes or no: Would a plastic bottle filled with air sink in water?",
        "Reasoning": "The density of air is much less than water. If the plastic bottle is filled with air, its overall density would be lower than water, causing it to float.",
        "Answer": "No"
    },
    {
        "Question": "Yes or no: Would a metal coin sink in water?",
        "Reasoning": "Metals typically have higher densities than water. Therefore, a metal coin would likely sink in water.",
        "Answer": "Yes"
    },
    {
        "Question": "Yes or no: Would a helium-filled balloon sink in water?",
        "Reasoning": "Helium is less dense than air, and a helium-filled balloon would be buoyant. It would rise in air and, in water, it would float.",
        "Answer": "No"
    }
]


example_prompt = PromptTemplate(input_variables=["Question","Reasoning" ,"Answer"], template=":Question:{Question}\n Reasoning:{Reasoning}\n Answer:{Answer}")


prompt = FewShotPromptTemplate(
      examples=examples,
      example_prompt=example_prompt,
      suffix="Question: {input}",
      input_variables=["input"]
  )

from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

chain = LLMChain(llm=ChatOpenAI(openai_api_key=api_key), prompt=prompt)

In [ ]:
chain.run('Yes or no: Would a plastic ball sink in water?')

'Reasoning: The density of a plastic ball is typically less than water. Therefore, a plastic ball would likely float in water.\nAnswer: No'

In [ ]:
#13.2 ReAct

from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI
import os

api_key = ''
serp_api = ''
os.environ['OPENAI_API_KEY'] = api_key
os.environ['SERPAPI_API_KEY']= serp_api

llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

agent_executor = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent_executor.invoke(
    {
        "input": "How to prepare Irani Tea famous in Hyderabad? explain from scratch"
    }
)



> Entering new AgentExecutor chain...
 I need to find a recipe
Action: Search
Action Input: Irani Tea recipe
Observation: ['Ingredients (US cup = 240ml ) · ▢ 1½ cups (360 ml) water · ▢ 2 tablespoons black tea (loose tea leaves or dust) · ▢ 6 green cardamoms · ▢ 1½ ...', 'Ingredients · 3 tablespoon Assam tea leaves · 1 cup Milk · 2 tablespoon Fresh cream · 1 Star anise · 2 Cardamom (Elaichi) Pods/Seeds · 1 Black ...', 'Take 1/3 cup of water in a pan. · Let the black tea mixture simmer for atleast 5 minutes on low flame. · Then add 500 ml of buffalo milk & turn up ...', '1/2 cup vermicelli (thin wheat noodles) · 1 liter full-fat milk · 1/2 cup sugar (adjust to taste) · 1/4 cup ghee (clarified butter) · 1/4 cup ...', 'Step 1 / 4 Boil the tea. In a saucepan, add 4 cups water, cardamom pods, cinnamon, tea powder, and sugar. Let it come to a boil.', 'Ingredients ; 3 cups water ; 1 1/2 tbsp black tea powder ; 3 cardamom pods optional but recommended ; 1 1/2 cup milk ; 2-3 heaped tbsp conden

{'input': 'How to prepare Irani Tea famous in Hyderabad? explain from scratch',
 'output': 'To make Irani Tea, boil 4 cups of water in a saucepan with cardamom pods, cinnamon, tea powder, and sugar. Let it come to a boil. Then add 2 cups of milk, 1/2 cup of condensed milk, and sugar to taste. Let it simmer for 5 minutes on low flame. Finally, add 1/2 teaspoon of pepper and serve hot.'}

In [ ]:
#13.3 Tree of Thoughts

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_experimental.smart_llm import SmartLLMChain

hard_question = "I have a 12 liter jug , a 4 liter jug and a 3 litre jug. I want to measure 6 liters. How do I do it?"
prompt = PromptTemplate.from_template(hard_question)
llm = ChatOpenAI(temperature=0)
chain = SmartLLMChain(llm=llm, prompt=prompt, n_ideas=3, verbose=True)
chain.run({})



> Entering new SmartLLMChain chain...
Prompt after formatting:
I have a 12 liter jug , a 4 liter jug and a 3 litre jug. I want to measure 6 liters. How do I do it?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainPendingDeprecationWarning: The function `from_strings` will be deprecated in a future version. Use from_messages classmethod instead.
  warn_deprecated(


Idea 1:
Step 1: Fill the 12 liter jug completely.
Step 2: Pour the contents of the 12 liter jug into the 4 liter jug. This leaves us with 8 liters in the 12 liter jug.
Step 3: Empty the 4 liter jug.
Step 4: Pour the remaining 8 liters from the 12 liter jug into the 4 liter jug. This fills up the 4 liter jug completely, leaving us with 8 liters in the 12 liter jug.
Step 5: Fill the 3 liter jug completely.
Step 6: Pour the contents of the 3 liter jug into the 4 liter jug. This fills up the 4 liter jug, leaving us with 1 liter in the 3 liter jug.
Step 7: Empty the 4 liter jug.
Step 8: Pour the 1 liter from the 3 liter jug into the 4 liter jug.
Step 9: Fill the 3 liter jug completely again.
Step 10: Pour the contents of the 3 liter jug into the 4 liter jug. This fills up the 4 liter jug completely, leaving us with 2 liters in the 3 liter jug.
Step 11: Pour the 2 liters from the 3 liter jug into the 12 liter jug.
Step 12: Fill the 3 liter jug completely again.
Step 13: Pour the contents of 

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainPendingDeprecationWarning: The function `from_strings` will be deprecated in a future version. Use from_messages classmethod instead.
  warn_deprecated(


Critique:
Idea 1: 
1. The step-by-step process provided in this answer option seems to be correct and does not contain any obvious flaws or faulty logic.

Idea 2: 
1. Step 5: Filling the 3 liter jug completely is not possible because the 3 liter jug is smaller than the 4 liter jug. This step is not feasible.
2. Step 6: Pouring the contents of the 3 liter jug into the 4 liter jug does not necessarily fill up the 4 liter jug completely. It depends on the amount of liquid in the 3 liter jug. This step assumes that the 4 liter jug will always be filled completely, which is not accurate.
3. Step 9: Filling the 3 liter jug completely again is not possible because the 3 liter jug is smaller than the 4 liter jug. This step is not feasible.
4. Step 10: Pouring the contents of the 3 liter jug into the 4 liter jug does not necessarily fill up the 4 liter jug completely. It depends on the amount of liquid in the 3 liter jug. This step assumes that the 4 liter jug will always be filled completely, 

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainPendingDeprecationWarning: The function `from_strings` will be deprecated in a future version. Use from_messages classmethod instead.
  warn_deprecated(


Resolution:
Step 1: Finding the researcher's preferred answer option:
Based on the information provided, it is not explicitly mentioned which of the three answer options the researcher thought was the best. Therefore, we cannot determine the researcher's preferred answer option.

Step 2: Improving the answer:
Since we do not have the researcher's preferred answer, we will consider improving Idea 1, which did not have any identified flaws or faulty logic.

Step 3: Printing the improved answer in full:
To measure 6 liters using a 12 liter jug, a 4 liter jug, and a 3 liter jug, follow these steps:

1. Fill the 12 liter jug completely.
2. Pour the contents of the 12 liter jug into the 4 liter jug. This leaves us with 8 liters in the 12 liter jug.
3. Empty the 4 liter jug.
4. Pour the remaining 8 liters from the 12 liter jug into the 4 liter jug. This fills up the 4 liter jug completely, leaving us with 8 liters in the 12 liter jug.
5. Fill the 3 liter jug completely.
6. Pour the contents o

"Step 1: Finding the researcher's preferred answer option:\nBased on the information provided, it is not explicitly mentioned which of the three answer options the researcher thought was the best. Therefore, we cannot determine the researcher's preferred answer option.\n\nStep 2: Improving the answer:\nSince we do not have the researcher's preferred answer, we will consider improving Idea 1, which did not have any identified flaws or faulty logic.\n\nStep 3: Printing the improved answer in full:\nTo measure 6 liters using a 12 liter jug, a 4 liter jug, and a 3 liter jug, follow these steps:\n\n1. Fill the 12 liter jug completely.\n2. Pour the contents of the 12 liter jug into the 4 liter jug. This leaves us with 8 liters in the 12 liter jug.\n3. Empty the 4 liter jug.\n4. Pour the remaining 8 liters from the 12 liter jug into the 4 liter jug. This fills up the 4 liter jug completely, leaving us with 8 liters in the 12 liter jug.\n5. Fill the 3 liter jug completely.\n6. Pour the content